In [3]:
import numpy as np
import h5py
import os
import re
import soundfile as sf
import glob
import mat73


In [31]:
room_list = ["bomb_shelter",
             "gym",
             "pb132",
             "pc226",
             "sa203",
             "sc203",
             "se203",
             "tb103",
             "tc352"]

sim_rir_path = '/scratch/ci411/DCASE_GEN/sim_rirs/'
out_rir_dir_path = '/scratch/ci411/DCASE_GEN/sim_rirs/'#'/vast/ci411/DCASE_GEN/sim_rirs/', will fix when i have vast approval

sr = 24000

for room in room_list:
    room_path = os.path.join(sim_rir_path, room)
    files = glob.glob(os.path.join(room_path, "*.hdf5"))
    files.sort()
    n_trajs, n_heights, _ = re.findall(r'\d+', files[-1].split('_')[-1])
    n_trajs = int(n_trajs)+1
    n_heights = int(n_heights)+1
    path_list = [[np.nan for j in range (n_heights)] for i in range(n_trajs)]
    print("Loading RIRs from {}".format(room))
    for file in files:
        print("Converting {}".format(file))
        filepath = os.path.join(room_path, file)
        with h5py.File(filepath, 'r+') as f:
            loaded_rirs = f['rirs'][()]
        loaded_rirs = np.moveaxis(loaded_rirs, [0,1,2],[2,0,1])
        
        out_rir_path = os.path.join(out_rir_dir_path, room)
        if not os.path.exists(out_rir_path):
            os.makedirs(out_rir_path)
        print("Saving to {}".format(out_rir_path))
        for i, rir in enumerate(loaded_rirs):
            rir_outpath = os.path.join(out_rir_path, file.split('.')[0]+f"i{i:03}"+'.wav')
            #sf.write(rir_outpath, rir, sr)

Loading RIRs from bomb_shelter
Converting /scratch/ci411/DCASE_GEN/sim_rirs/bomb_shelter/bomb_shelter_t0h0.hdf5
Saving to /vast/ci411/DCASE_GEN/sim_rirs/bomb_shelter
Converting /scratch/ci411/DCASE_GEN/sim_rirs/bomb_shelter/bomb_shelter_t0h1.hdf5
Saving to /vast/ci411/DCASE_GEN/sim_rirs/bomb_shelter
Converting /scratch/ci411/DCASE_GEN/sim_rirs/bomb_shelter/bomb_shelter_t0h2.hdf5
Saving to /vast/ci411/DCASE_GEN/sim_rirs/bomb_shelter
Converting /scratch/ci411/DCASE_GEN/sim_rirs/bomb_shelter/bomb_shelter_t0h3.hdf5
Saving to /vast/ci411/DCASE_GEN/sim_rirs/bomb_shelter
Converting /scratch/ci411/DCASE_GEN/sim_rirs/bomb_shelter/bomb_shelter_t0h4.hdf5
Saving to /vast/ci411/DCASE_GEN/sim_rirs/bomb_shelter
Converting /scratch/ci411/DCASE_GEN/sim_rirs/bomb_shelter/bomb_shelter_t0h5.hdf5
Saving to /vast/ci411/DCASE_GEN/sim_rirs/bomb_shelter


KeyboardInterrupt: 

In [4]:
tau_rir_path = '/scratch/ci411/TAU_SRIR_DB/TAU-SRIR_DB/'
tau_rir_outpath = '/scratch/ci411/DCASE_GEN/tau_srir'

tau_files = glob.glob(os.path.join(tau_rir_path, 'rirs*'))
tau_files.sort()
sr = 24000

for room_idx, tau_file in enumerate(tau_files):
    print(f"Loading from {tau_file}")
    rirs_loaded = mat73.loadmat(tau_file)
    room = room_list[room_idx]
    print(f"Processing room {room}")
    for aud_format in ['mic', 'foa']:
        rirs = rirs_loaded['rirs'][aud_format]
        print(f"Storing results in {out_dirs}")
        out_dirs = os.path.join(tau_rir_outpath, room, aud_format)         
        if not os.path.exists(out_dirs):
            os.makedirs(out_dirs)
        for traj_idx in range(len(rirs)):
            for h_idx in range(len(rirs[0])):
                rir_array = rirs[traj_idx][h_idx]
                rir_array = np.moveaxis(rir_array, [0,1,2],[1,2,0])
                
                for i, rir in enumerate(rir_array):
                    filename = f"{room}_t{traj_idx}h{h_idx}i{i:03}.wav"
                    rir_outpath = os.path.join(tau_rir_outpath, room, aud_format, filename)
                    #sf.write(rir_outpath, rir, sr)

Loading from /scratch/ci411/TAU_SRIR_DB/TAU-SRIR_DB/rirs_01_bomb_shelter.mat


KeyboardInterrupt: 

In [7]:
test_tau_file = '/scratch/ci411/TAU_SRIR_DB/TAU-SRIR_DB/rirs_01_bomb_shelter.mat'
rirs_loaded = mat73.loadmat(tau_file)['rirs']['mic']


In [12]:
rirs_loaded[0][0].shape

(7200, 4, 360)